In [84]:
import os

In [85]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [86]:
os.chdir("../")

In [87]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [88]:
project_path = "C:/Users/RICH-FILES/Desktop/ml/Loan-Amount-Prediction"

os.chdir(project_path)

In [89]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: float
    

In [90]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories




In [91]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
        root_dir = config.root_dir,
        train_data_path=config.train_data_path,
        test_data_path=config.test_data_path,
        model_name = config.model_name,
        alpha=params.alpha,
        l1_ratio=params.l1_ratio,
        target_column=schema.name
        )
        
        return model_trainer_config
    
   
        
        
    
        

In [92]:

import os
from credit_risk import logger
from sklearn.linear_model import ElasticNet
import joblib
import pandas as pd


In [93]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        test_data.dropna(inplace=True)
        train_data.dropna(inplace=True)
        
        print(test_data.isnull().sum())
        print(train_data.isnull().sum())
        
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop(self.config.target_column, axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        
        elnet   = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        elnet.fit(train_x, train_y)
       
        joblib.dump(elnet,os.path.join(self.config.root_dir, self.config.model_name))

In [94]:
#define pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e    

[2025-03-09 20:53:01,975: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-09 20:53:01,978: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-09 20:53:01,981: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-09 20:53:01,982: INFO: common: created directory at: artifacts]
[2025-03-09 20:53:01,984: INFO: common: created directory at: artifacts/model_trainer]
Age                         0
Income                      0
Emp_length                  0
Amount                      0
Rate                        0
Percent_income              0
Home_MORTGAGE               0
Home_OTHER                  0
Home_OWN                    0
Home_RENT                   0
Intent_DEBTCONSOLIDATION    0
Intent_EDUCATION            0
Intent_HOMEIMPROVEMENT      0
Intent_MEDICAL              0
Intent_PERSONAL             0
Intent_VENTURE              0
dtype: int64
Age                         0
Income                      0
Emp_length                  